# <font color='blue'> Data Sprint HDB Resale Prices </font>

Group 4: Ameer, Benjamin, Leonard, Wei Yu.

### Exploration:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# set the plots to display in the Jupyter notebook
%matplotlib inline